In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv'):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

In [4]:
num_iteration = 10
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

for iteration in range(num_iteration):
    print(f"{iteration+1}th iteration")
    for history_size in history_list:
        for future_size in future_list:
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration+1
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists("./model", model_name, 'model'):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}")
            else:
                DATT_seq2seq_GRU.model = loadfile("./model", model_name, 'model')
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists('./result', model_name):
                savefile(test_result, './result', model_name)
            print("end training")
            print("")
    print("\n\n")

1th iteration
history size: 10
future size: 10
model is loaded from: ./model/10_10_1_50_1_50_datt_seq2seq_gru_1


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.900671,0.947878,0.969716,0.939422,5.117202,3.992389,2.71327,3.940954
1,0.799622,0.894724,0.944357,0.879568,7.221528,5.674612,3.676933,5.524358
2,0.676867,0.829835,0.906914,0.804539,9.045753,7.215126,4.754602,7.00516
3,0.551501,0.78096,0.868805,0.733755,10.515697,8.187118,5.643127,8.115314
4,0.433028,0.74443,0.835415,0.670957,11.707734,8.844738,6.318536,8.957003
5,0.319931,0.717127,0.805756,0.614272,12.698957,9.306761,6.862569,9.622762
6,0.215111,0.696835,0.778403,0.56345,13.51328,9.635625,7.328513,10.159139
7,0.122681,0.682295,0.756525,0.5205,14.252891,9.864339,7.681024,10.599418
8,0.045372,0.671004,0.741679,0.486019,14.884819,10.038568,7.911127,10.944838


end training

history size: 10
future size: 20
model is loaded from: ./model/10_20_1_50_1_50_datt_seq2seq_gru_1


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.877108,0.919942,0.941436,0.912829,6.924128,4.943281,3.765284,5.210898
1,0.823525,0.879394,0.917997,0.873639,8.146417,6.069187,4.455002,6.223535
2,0.735699,0.845673,0.882344,0.821239,9.534244,6.867994,5.335568,7.245935
3,0.630755,0.816324,0.842684,0.763254,10.892952,7.49522,6.168187,8.185453
4,0.523513,0.789441,0.809465,0.707473,12.176909,8.028594,6.786194,8.997232
5,0.419288,0.766111,0.787075,0.657491,13.398944,8.465843,7.17125,9.678679
6,0.321144,0.74225,0.77384,0.612411,14.204363,8.8899,7.38873,10.160998
7,0.231333,0.715846,0.764961,0.570713,14.825685,9.337125,7.530679,10.564497
8,0.151558,0.688594,0.755916,0.532023,15.34495,9.777842,7.672545,10.931779


end training

history size: 10
future size: 30
Epoch 1/10000
572/572 - 8s - loss: 0.3846 - val_loss: 0.3025 - 8s/epoch - 14ms/step
Epoch 2/10000
572/572 - 5s - loss: 0.2971 - val_loss: 0.2776 - 5s/epoch - 8ms/step
Epoch 3/10000
572/572 - 4s - loss: 0.2764 - val_loss: 0.2656 - 4s/epoch - 8ms/step
Epoch 4/10000
572/572 - 5s - loss: 0.2599 - val_loss: 0.2521 - 5s/epoch - 8ms/step
Epoch 5/10000
572/572 - 6s - loss: 0.2470 - val_loss: 0.2455 - 6s/epoch - 10ms/step
Epoch 6/10000
572/572 - 5s - loss: 0.2359 - val_loss: 0.2326 - 5s/epoch - 8ms/step
Epoch 7/10000
572/572 - 4s - loss: 0.2279 - val_loss: 0.2245 - 4s/epoch - 8ms/step
Epoch 8/10000
572/572 - 4s - loss: 0.2204 - val_loss: 0.2144 - 4s/epoch - 8ms/step
Epoch 9/10000
572/572 - 5s - loss: 0.2129 - val_loss: 0.2212 - 5s/epoch - 8ms/step
Epoch 10/10000
572/572 - 5s - loss: 0.2064 - val_loss: 0.2101 - 5s/epoch - 9ms/step
Epoch 11/10000
572/572 - 6s - loss: 0.1978 - val_loss: 0.2055 - 6s/epoch - 11ms/step
Epoch 12/10000
572/572 - 5s - loss:

...

INFO:tensorflow:Assets written to: ./model/40_20_1_50_1_50_datt_seq2seq_gru_1\assets


INFO:tensorflow:Assets written to: ./model/40_20_1_50_1_50_datt_seq2seq_gru_1\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.913741,0.89525,0.93191,0.913633,5.795058,5.69088,4.008191,5.16471
1,0.841715,0.765577,0.876962,0.828085,7.709077,8.516104,5.389555,7.204912
2,0.744795,0.701873,0.796194,0.747621,9.362351,9.607465,6.93855,8.636122
3,0.640476,0.669654,0.720539,0.67689,10.741177,10.116872,8.126747,9.661598
4,0.535155,0.650142,0.666357,0.617218,12.018037,10.415991,8.881679,10.438569
5,0.435026,0.639643,0.637443,0.570704,13.204613,10.575908,9.26015,11.013557
6,0.342984,0.635663,0.625937,0.534861,13.960408,10.63657,9.407852,11.334943
7,0.263234,0.633234,0.621338,0.505935,14.500798,10.674354,9.467354,11.547502
8,0.194559,0.626066,0.611696,0.47744,14.937544,10.780037,9.588844,11.768808


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_1.csv
end training

history size: 40
future size: 30
Epoch 1/10000
559/559 - 8s - loss: 0.3750 - val_loss: 0.3110 - 8s/epoch - 14ms/step
Epoch 2/10000
559/559 - 5s - loss: 0.2868 - val_loss: 0.2918 - 5s/epoch - 8ms/step
Epoch 3/10000
559/559 - 5s - loss: 0.2640 - val_loss: 0.2640 - 5s/epoch - 9ms/step
Epoch 4/10000
559/559 - 4s - loss: 0.2461 - val_loss: 0.2445 - 4s/epoch - 8ms/step
Epoch 5/10000
559/559 - 5s - loss: 0.2295 - val_loss: 0.2346 - 5s/epoch - 8ms/step
Epoch 6/10000
559/559 - 5s - loss: 0.2153 - val_loss: 0.2236 - 5s/epoch - 8ms/step
Epoch 7/10000
559/559 - 4s - loss: 0.2029 - val_loss: 0.2069 - 4s/epoch - 8ms/step
Epoch 8/10000
559/559 - 5s - loss: 0.1937 - val_loss: 0.2026 - 5s/epoch - 8ms/step
Epoch 9/10000
559/559 - 5s - loss: 0.1839 - val_loss: 0.1923 - 5s/epoch - 8ms/step
Epoch 10/10000
559/559 - 5s - loss: 0.1753 - val_loss: 0.1786 - 5s/epoch - 9ms/step
Epoch 11/10000
559/559 - 6s - loss: 0.1722 - val_lo

KeyboardInterrupt: 